In [1]:
# json_inference_consumer.py

from kafka import KafkaConsumer, KafkaProducer
import os
import json
import uuid
from concurrent.futures import ThreadPoolExecutor


In [2]:

TOPIC_NAME = "avg-data"

KAFKA_SERVER = "kafka:29092"

NOTIFICATION_TOPIC = "NOTIFICATION"


In [3]:

consumer = KafkaConsumer(
    TOPIC_NAME,
    bootstrap_servers=[KAFKA_SERVER],
    auto_offset_reset='earliest',
    enable_auto_commit=False,
    # to deserialize kafka.producer.object into dict
    #value_deserializer=lambda m: json.loads(m.decode('utf-8')),
)


In [4]:
producer = KafkaProducer(
    bootstrap_servers = KAFKA_SERVER,
    api_version = (0, 11, 15)
)

In [5]:
def inferenceProcessFunction(data):
    # process steps with data in json format      
    
    json_data = json.loads(data)
    
    print("eventTime:" + str(json_data['eventTime']) + ' / ' + "nome:" + str(json_data['nome']) + ' / ' + "bpm:" + str(json_data['avgbpm']) + ' / ' + "temp:" + str(json_data['avgtemp']))
    
    if(json_data['avgbpm'] >= 120 or json_data['avgtemp'] >= 39):
        notification_data = data        
        producer.send(NOTIFICATION_TOPIC, notification_data)
        producer.flush()
        print('\nNotification Topic Sent!\n')
    

In [6]:
for inf in consumer:
    print('processing line...')  
    print(inf.value)
    inf_data = inf.value
    inferenceProcessFunction(inf_data)

processing line...
b'{"eventTime":"2022-03-01 00:25:00","nome":"maria","avgbpm":100.0,"avgtemp":34.0}'
eventTime:2022-03-01 00:25:00 / nome:maria / bpm:100.0 / temp:34.0
processing line...
b'{"eventTime":"2022-03-01 02:35:00","nome":"joao","avgbpm":110.0,"avgtemp":39.0}'
eventTime:2022-03-01 02:35:00 / nome:joao / bpm:110.0 / temp:39.0

Notification Topic Sent!

processing line...
b'{"eventTime":"2022-03-01 04:35:00","nome":"antonio","avgbpm":72.0,"avgtemp":38.0}'
eventTime:2022-03-01 04:35:00 / nome:antonio / bpm:72.0 / temp:38.0
processing line...
b'{"eventTime":"2022-03-01 06:40:00","nome":"maria","avgbpm":57.0,"avgtemp":38.0}'
eventTime:2022-03-01 06:40:00 / nome:maria / bpm:57.0 / temp:38.0
processing line...
b'{"eventTime":"2022-03-01 06:55:00","nome":"joao","avgbpm":74.0,"avgtemp":34.0}'
eventTime:2022-03-01 06:55:00 / nome:joao / bpm:74.0 / temp:34.0
processing line...
b'{"eventTime":"2022-03-01 08:15:00","nome":"antonio","avgbpm":85.0,"avgtemp":36.0}'
eventTime:2022-03-01 08:1

KeyboardInterrupt: 